## Setup: Finnhub API Key

This notebook uses real market data from [Finnhub](https://finnhub.io/) (free tier: 60 API calls/min).

**To use real data:**
1. Get a free API key at https://finnhub.io/register
2. Set environment variable: `export FINNHUB_API_KEY=your_key_here`
3. Or enter the key when prompted

**Without a key:** The notebook falls back to synthetic data generation.

# Hawkes Process
Conditional intensity: $\lambda(t)=\mu + \sum_{t_i<t} \alpha e^{-\beta (t-t_i)}$.

In [ ]:
import numpy as np, matplotlib.pyplot as plt

# Import Finnhub helper for real market data
try:
    from python.finnhub_helper import fetch_historical_simulation, get_finnhub_api_key, create_orderbook_from_quote
    FINNHUB_AVAILABLE = True
except Exception as e:
    print(f"Finnhub helper not available: {e}")
    FINNHUB_AVAILABLE = False

np.random.seed(7)
T=6000
mu=0.8; alpha=0.6; beta=2.0
# simulate via thinning (approx)
t=[0.0]
while len(t)<T:
    lam=mu+alpha*sum(np.exp(-beta*(t[-1]-ti)) for ti in t)
    w=np.random.exponential(1/max(lam,1e-6))
    t_next=t[-1]+w
    # accept with prob lam / (mu+alpha)
    if np.random.rand()<min(1.0, lam/max(mu+alpha,1e-6)):
        t.append(t_next)
arr=np.array(t)


## Fit with tick if available (fallback otherwise)

In [ ]:
try:
    from tick.hawkes import HawkesExpKern
    model=HawkesExpKern(decays=beta)
    model.fit([arr])
    print('baseline',model.baseline)
except Exception:
    pass
ia=np.diff(arr)
plt.figure(); plt.hist(ia, bins=50); plt.title('Interarrivals'); plt.show()
